# Predicción de Reservas y Análisis de Rentabilidad para OilyGiant

**Objetivo:** Ayudar a la compañía minera OilyGiant a encontrar la mejor ubicación para el desarrollo de nuevos pozos de petróleo. Para ello, construiremos un modelo de regresión lineal para predecir el volumen de reservas, calcularemos las ganancias potenciales y analizaremos los riesgos utilizando la técnica de bootstrapping.

## 1. Carga y Preparación de Datos

### 1.1. Importación de Librerías y Definición de Constantes

Primero, importaremos todas las librerías necesarias y definiremos las constantes clave del proyecto para mantener el código limpio y fácil de modificar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Constantes del proyecto basadas en las condiciones
BUDGET = 100_000_000  # Presupuesto para 200 pozos ($100M)
WELLS_TO_STUDY = 500  # Puntos estudiados en la exploración
BEST_WELLS_TO_DEVELOP = 200 # Puntos seleccionados para el desarrollo
REVENUE_PER_UNIT = 4500  # Ingreso por unidad de producto ($4500 por mil barriles)
LOSS_RISK_THRESHOLD = 2.5  # Umbral de riesgo de pérdidas permitido (%)
BOOTSTRAP_SAMPLES = 1000 # Número de muestras para bootstrapping

# Ignorar advertencias para una salida más limpia
import warnings
warnings.filterwarnings('ignore')

### 1.2. Carga y Revisión de Datos

Cargamos los datos de las tres regiones y realizamos una inspección inicial para entender su estructura, verificar tipos de datos y buscar valores ausentes.

In [ ]:
# Cargar los datos de las tres regiones
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')

# Guardar los dataframes en una lista para iterar fácilmente
regions_df = [df0, df1, df2]
region_names = ['Región 0', 'Región 1', 'Región 2']

# Inspeccionar los datos de cada región
for i, df in enumerate(regions_df):
    print(f"--- Información para la {region_names[i]} ---")
    print("\nPrimeras 5 filas:")
    display(df.head())
    print("\nInformación general:")
    df.info()
    print("\nEstadísticas descriptivas:")
    display(df.describe())
    print("\nSuma de valores nulos:")
    print(df.isnull().sum())
    print("\nNúmero de duplicados:", df.duplicated().sum())
    print("-" * 50, "\n")

--- Información para la Región 0 ---

Primeras 5 filas:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Estadísticas descriptivas:


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Suma de valores nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Número de duplicados: 0
-------------------------------------------------- 

--- Información para la Región 1 ---

Primeras 5 filas:


...


Suma de valores nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Número de duplicados: 0
-------------------------------------------------- 

--- Información para la Región 2 ---

Primeras 5 filas:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Estadísticas descriptivas:


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Suma de valores nulos:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Número de duplicados: 0
-------------------------------------------------- 



#### Conclusión de la Preparación de Datos

* Los datos de las tres regiones han sido cargados correctamente.
* Cada conjunto de datos contiene 100,000 filas y 5 columnas: `id`, `f0`, `f1`, `f2`, y `product`.
* No se encontraron valores nulos ni filas duplicadas en ninguno de los conjuntos de datos.
* La columna `id` es un identificador único y no debe ser utilizada para el entrenamiento del modelo. Las características son `f0`, `f1` y `f2`, y nuestro objetivo a predecir es `product`.
* Los datos están listos para la siguiente fase de entrenamiento del modelo.

## 2. Entrenamiento y Evaluación de Modelos

Ahora entrenaremos un modelo de regresión lineal para cada región, lo evaluaremos y guardaremos las predicciones para los cálculos posteriores.

In [ ]:
# Lista para guardar los resultados de cada región (predicciones y valores reales)
results = []

print("--- Resultados del Entrenamiento y Evaluación por Región ---\n")

for i, df in enumerate(regions_df):
    # 1. Definir características (X) y objetivo (y)
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']

    # 2. Dividir los datos en 75% entrenamiento y 25% validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42
    )

    # 3. Entrenar el modelo de Regresión Lineal
    model = LinearRegression()
    model.fit(features_train, target_train)

    # 4. Realizar predicciones en el conjunto de validación
    predictions_valid = model.predict(features_valid)

    # 5. Calcular métricas: RMSE y volumen promedio predicho
    rmse = mean_squared_error(target_valid, predictions_valid, squared=False)
    avg_pred_volume = predictions_valid.mean()

    # Imprimir resultados
    print(f"{region_names[i]}:")
    print(f"  - Volumen promedio de reservas predicho: {avg_pred_volume:.2f} (miles de barriles)")
    print(f"  - RECM del modelo: {rmse:.2f}\n")

    # 6. Guardar los resultados para los siguientes pasos
    results.append({
        'name': region_names[i],
        'target_valid': target_valid.reset_index(drop=True),
        'predictions_valid': pd.Series(predictions_valid)
    })

--- Resultados del Entrenamiento y Evaluación por Región ---

Región 0:
  - Volumen promedio de reservas predicho: 92.40 (miles de barriles)
  - RECM del modelo: 37.76

Región 1:
  - Volumen promedio de reservas predicho: 68.71 (miles de barriles)
  - RECM del modelo: 0.89

Región 2:
  - Volumen promedio de reservas predicho: 94.77 (miles de barriles)
  - RECM del modelo: 40.15



#### Análisis de los Resultados del Modelo

* **Región 0 y Región 2:** Muestran un RECM (Error Cuadrático Medio Raíz) relativamente alto (37.76 y 40.15 respectivamente), lo que indica una mayor incertidumbre en las predicciones. Sin embargo, su volumen promedio de reservas predicho es considerablemente alto (92.40 y 94.77).
* **Región 1:** Tiene un RECM extremadamente bajo (0.89), lo que sugiere que el modelo es muy preciso para esta región. No obstante, el volumen promedio de reservas predicho es más bajo (68.71) en comparación con las otras dos regiones.

## 3. Preparación para el Cálculo de Ganancias

Calcularemos el volumen de reservas necesario para que el desarrollo de un pozo no genere pérdidas y lo compararemos con el promedio de cada región.

In [ ]:
# Costo de desarrollo por pozo
cost_per_well = BUDGET / BEST_WELLS_TO_DEVELOP

# Volumen de reservas de equilibrio (breakeven) por pozo
breakeven_volume_per_well = cost_per_well / REVENUE_PER_UNIT

print(f"Presupuesto: ${BUDGET:,.2f}")
print(f"Ingreso por mil barriles: ${REVENUE_PER_UNIT:,.2f}")
print(f"Costo por pozo: ${cost_per_well:,.2f}")
print(f"Volumen de equilibrio por pozo: {breakeven_volume_per_well:.2f} (miles de barriles)\n")

# Comparar con el volumen medio real de cada región
for i, df in enumerate(regions_df):
    avg_real_volume = df['product'].mean()
    print(f"Volumen promedio real en {region_names[i]}: {avg_real_volume:.2f} (miles de barriles)")
    if avg_real_volume < breakeven_volume_per_well:
        print("  -> El volumen promedio es INSUFICIENTE para cubrir los costos.\n")
    else:
        print("  -> El volumen promedio es SUFICIENTE para cubrir los costos.\n")

Presupuesto: $100,000,000.00
Ingreso por mil barriles: $4,500.00
Costo por pozo: $500,000.00
Volumen de equilibrio por pozo: 111.11 (miles de barriles)

Volumen promedio real en Región 0: 92.50 (miles de barriles)
  -> El volumen promedio es INSUFICIENTE para cubrir los costos.

Volumen promedio real en Región 1: 68.83 (miles de barriles)
  -> El volumen promedio es INSUFICIENTE para cubrir los costos.

Volumen promedio real en Región 2: 95.00 (miles de barriles)
  -> El volumen promedio es INSUFICIENTE para cubrir los costos.



#### Conclusión sobre el Cálculo de Ganancias

* Para que un pozo sea rentable, necesita tener un volumen de reservas de al menos **111.11 mil barriles**.
* Ninguna de las tres regiones tiene un volumen de reservas promedio que supere este umbral de rentabilidad.
* Esto confirma la necesidad de utilizar el modelo de machine learning para seleccionar solo los pozos con las mayores reservas estimadas, en lugar de perforar al azar.

## 4. Función para Calcular la Ganancia

Escribiremos una función que calcule la ganancia total de los 200 mejores pozos seleccionados según las predicciones del modelo.

In [ ]:
def calculate_profit(target, predictions, count):
    """
    Calcula la ganancia para los `count` mejores pozos seleccionados por `predictions`.

    Args:
        target (pd.Series): Volumen real de reservas.
        predictions (pd.Series): Volumen predicho de reservas.
        count (int): Número de pozos a seleccionar.

    Returns:
        float: Ganancia total.
    """
    # Escoger los pozos con las predicciones más altas
    top_predictions_indices = predictions.sort_values(ascending=False).index
    top_wells_indices = top_predictions_indices[:count]

    # Resumir el volumen objetivo de los mejores pozos
    selected_target_volume = target.loc[top_wells_indices].sum()

    # Calcular la ganancia
    profit = (selected_target_volume * REVENUE_PER_UNIT) - BUDGET
    return profit

## 5. Cálculo de Riesgos y Ganancias con Bootstrapping

Utilizaremos la técnica de bootstrapping para simular 1000 veces el proceso de selección y cálculo de ganancias. Esto nos permitirá encontrar la distribución de beneficios, el intervalo de confianza del 95% y el riesgo de pérdidas para cada región.

In [ ]:
print("--- Análisis de Riesgos y Ganancias por Región ---\n")

final_results = []

for result in results:
    target = result['target_valid']
    predictions = result['predictions_valid']
    region_name = result['name']

    profits = []

    # Iniciar el bootstrapping con 1000 muestras
    # Usamos np.random.RandomState() para asegurar la reproducibilidad dentro del bucle
    state = np.random.RandomState(42)

    for _ in range(BOOTSTRAP_SAMPLES):
        # Crear una submuestra de 500 puntos con reemplazo
        target_subsample = target.sample(n=WELLS_TO_STUDY, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index]

        # Calcular la ganancia para los 200 mejores pozos de la submuestra
        profit = calculate_profit(target_subsample, predictions_subsample, BEST_WELLS_TO_DEVELOP)
        profits.append(profit)

    # Convertir a Series para facilitar los cálculos
    profits = pd.Series(profits)

    # Calcular métricas finales
    avg_profit = profits.mean()
    confidence_interval = (profits.quantile(0.025), profits.quantile(0.975))
    loss_risk = (profits < 0).mean() * 100 # Probabilidad de pérdida en porcentaje

    # Guardar resultados
    final_results.append({
        'Región': region_name,
        'Ganancia Promedio (USD)': avg_profit,
        'Intervalo de Confianza 95%': confidence_interval,
        'Riesgo de Pérdidas (%)': loss_risk
    })

    # Imprimir hallazgos
    print(f"{region_name}:")
    print(f"  - Ganancia Promedio: ${avg_profit:,.2f}")
    print(f"  - Intervalo de Confianza 95%: (${confidence_interval[0]:,.2f}, ${confidence_interval[1]:,.2f})")
    print(f"  - Riesgo de Pérdidas: {loss_risk:.2f}%\n")

# Convertir los resultados finales a un DataFrame para una mejor visualización
final_df = pd.DataFrame(final_results)
display(final_df)

--- Análisis de Riesgos y Ganancias por Región ---

Región 0:
  - Ganancia Promedio: $6,061,226.32
  - Intervalo de Confianza 95%: ($100,894.12, $12,463,709.81)
  - Riesgo de Pérdidas: 2.50%

Región 1:
  - Ganancia Promedio: $6,651,176.54
  - Intervalo de Confianza 95%: ($1,808,515.85, $12,057,104.61)
  - Riesgo de Pérdidas: 0.20%

Región 2:
  - Ganancia Promedio: $5,851,036.38
  - Intervalo de Confianza 95%: ($-8,369.42, $12,120,508.98)
  - Riesgo de Pérdidas: 2.60%



,Región,Ganancia Promedio (USD),Intervalo de Confianza 95%,Riesgo de Pérdidas (%)
0,Región 0,6.061226e+06,"(100894.11995841737, 12463709.805645682)",2.5
1,Región 1,6.651177e+06,"(1808515.8498991863, 12057104.608184986)",0.2
2,Región 2,5.851036e+06,"(-8369.423215681249, 12120508.982598253)",2.6


## 6. Conclusión y Recomendación Final

Basándonos en el análisis de riesgos y ganancias, podemos seleccionar la región más prometedora para el desarrollo.

#### Hallazgos Clave

1.  **Filtrado por Riesgo:** La condición del negocio establece que solo debemos considerar regiones con un riesgo de pérdidas inferior al 2.5%.
    * Región 0: Riesgo de 2.50% (NO CUMPLE, el riesgo debe ser estrictamente inferior).
    * Región 1: Riesgo de 0.20% (ACEPTABLE).
    * Región 2: Riesgo de 2.60% (DEMASIADO ALTO).

    Basado en este criterio, las regiones 0 y 2 deben ser descartadas. La única región que cumple con el requisito de bajo riesgo es la **Región 1**.

2.  **Selección por Ganancia:** De las regiones que cumplen con el criterio de riesgo, debemos seleccionar la que tenga la mayor ganancia promedio. Dado que solo la Región 1 califica, se convierte automáticamente en la región recomendada.

#### Recomendación

Se recomienda desarrollar los nuevos pozos de petróleo en la **Región 1**.

**Justificación:**

* **Riesgo Mínimo:** La Región 1 es la única que presenta un riesgo de pérdidas (0.20%) inferior al umbral aceptable del 2.5%. Esto la convierte en la opción más segura desde una perspectiva de inversión.
* **Ganancia Positiva Confiable:** Su ganancia promedio es de `$6,651,176.54` y el intervalo de confianza del 95% se encuentra completamente en territorio positivo, lo que refuerza la viabilidad financiera del proyecto en esta región.

Aunque las otras regiones mostraron un potencial de reservas promedio más alto en el análisis inicial, la alta incertidumbre de sus modelos (RECM alto) se traduce en un riesgo financiero inaceptable. Por lo tanto, la Región 1 representa el equilibrio óptimo entre rentabilidad y seguridad para la inversión de OilyGiant.